In [2]:
from NFR2 import NFR
from PIL import Image

In [ ]:
image_path = "D://Retinafacetest/h.jpg"
name, xmin, ymin, w, h, xmax, ymax,dis = NFR.FindFaceFromImage(image_path,showlog=True)

print(f"Employee name: {name} \ncoordinates: {xmin},{ymin},{w},{h},{xmax},{ymax}\ndistance = {dis}")

yooo
**********************************
[                identity                                      hash  target_x  \
0  DatabaseFR\hamza1.png  cc01ee63c672464aa33e6452d82ae10443b43c02        96   

   target_y  target_w  target_h  source_x  source_y  source_w  source_h  \
0       259       572       572       106       253       246       246   

   threshold  distance  
0        0.4  0.339913  ]
Employee name: hamza1 
coordinates: 106,253,246,246,352,499
distance = 0.33991251920698085


## **new input**


In [2]:
# - input : image with bounding boxes array 
# - output: new croped images


In [47]:
import glob
import os


def crop_images(image_path, bounding_boxes):
    cropped_images = []
    # to clear all of the folder past contents (we dont need the previous croped images right?)
    files = glob.glob(os.path.join("Croppedimages", '*'))
    for f in files:
        os.remove(f)
    with Image.open(image_path) as img:
        for box in bounding_boxes:
            x, y, w, h = box
            crop_box = (x, y, x + w, y + h)
            cropped_img = img.crop(crop_box)
            cropped_images.append((cropped_img,(x,y,w,h)))
            # cropped_img.save(os.path.join("Croppedimages/", f"cropped_image_{i}.jpg"))
            filename = f"cropped_image.jpg"
            new_image_path = os.path.join("Croppedimages/", filename)
            increment = 1
            
            while os.path.exists(new_image_path):
                new_image_path = os.path.join("Croppedimages/", f"cropped_image_{increment}.jpg")
                increment += 1
                
            # Save the cropped image
            cropped_img.save(new_image_path)
    print(cropped_images)
    return cropped_images

## *a test code for the crop*

In [50]:
bounding_boxes = [(62,237,212,38), (300, 300, 150, 150)]
cropped_imgs = crop_images('D://Retinafacetest/three.jpg',bounding_boxes)
print(cropped_imgs[1][0])

new_images=[]



[(<PIL.Image.Image image mode=RGB size=212x38 at 0x2016A577200>, (62, 237, 212, 38)), (<PIL.Image.Image image mode=RGB size=150x150 at 0x2016A5770E0>, (300, 300, 150, 150))]
<PIL.Image.Image image mode=RGB size=150x150 at 0x2016A5770E0>


## **New function call**

In [6]:
# new_images.append(f"cropped_image2.jpg")
# print(new_images)
# new_images2=[("cropped_image0.jpg",(62,237,212,38)),("cropped_image1.jpg",(300, 300, 150, 150)),("cropped_image2.jpg",(4,237,212,38))]
# def FaceRec(showlog):
#     findings= list([],)
#     for i in new_images2:
#         print(i)
#         print("\n----------------------------------------------\n")
#         print(f"Image {i} : \n")
#         newfindings=NFR.FindFaceFromImage(i,showlog=True)  
#         print(f"neew finding:\n{newfindings}\n")
#         findings.append(newfindings)
#         print("\n----------------------------------------------\n")
#     return findings


# with out pp
def FaceRec(new_images2, showlog):
    updated_images = []
    
    for i, (image_path, coords) in enumerate(new_images2):
        print(image_path)
        print("\n----------------------------------------------\n")
        print(f"Image {image_path} : \n")
        
        # Assume NFR.FindFaceFromImage takes an image path and a showlog parameter
        newfindings = NFR.FindFaceFromImage(image_path, showlog=showlog)
        
        print(f"new finding:\n{newfindings}\n")
        updated_images.append((image_path, coords, newfindings))
        
        print("\n----------------------------------------------\n")
    
    return updated_images


In [12]:
new_images2 = [
    # ("cropped_image0.jpg", (62, 237, 212, 38)),
    # ("cropped_image1.jpg", (300, 300, 150, 150)),
    # ("cropped_image2.jpg", (4, 237, 212, 38)),
    # ("D://Retinafacetest/ms.jpg", (4, 237, 212, 38)),
    # ("D://Retinafacetest/dar.jpg", (4, 237, 212, 38)),
    ("D://Retinafacetest/three1.jpg", (4, 237, 212, 38)),
    # ("D://Retinafacetest/h.jpg", (4, 237, 212, 38)),
    
    
    
]
showlog = False
updated_images2 = FaceRec(new_images2, showlog)
print(updated_images2)

D://Retinafacetest/three1.jpg

----------------------------------------------

Image D://Retinafacetest/three1.jpg : 

24-07-03 18:30:07 - Found 7 newly added image(s), 7 removed image(s), 0 replaced image(s).


Finding representations:   0%|          | 0/7 [00:00<?, ?it/s]

Finding representations: 100%|██████████| 7/7 [00:14<00:00,  2.05s/it]


24-07-03 18:30:21 - There are now 8 representations in Database.pkl
24-07-03 18:30:21 - Searching D://Retinafacetest/three1.jpg in 8 length datastore
24-07-03 18:30:23 - find function duration 15.633016347885132 seconds
new finding:
['Noore', 28, 87, 162, 162, 190, 249, 0.2882234397537323]


----------------------------------------------

[('D://Retinafacetest/three1.jpg', (4, 237, 212, 38), ['Noore', 28, 87, 162, 162, 190, 249, 0.2882234397537323])]


In [5]:
# Deepface with pp
import concurrent.futures
import tensorflow as tf
import numpy as np

# def load_and_preprocess_image(image_path):
#     # Load image using TensorFlow
#     image = tf.io.read_file(image_path)
#     image = tf.image.decode_image(image)
#     # Preprocess image if necessary (resize, normalize, etc.)
#     image = tf.image.resize(image, [224, 224])  # Example resize to 224x224
#     image = image / 255.0  # Example normalization
#     return image

def process_image(image_data, silent):
    image_path, coords = image_data
    if not silent:
        print("\n----------------------------------------------\n")
        print(f"Image {image_path} : \n")
        newfindings = NFR.FindFaceFromImage(image_path, silent=silent)
        print(f"new finding:\n{newfindings}\n")
        print("\n----------------------------------------------\n")
    else : 
        newfindings = NFR.FindFaceFromImage(image_path,silent=silent)
    return (image_path, coords, newfindings)

def FaceRecPP(new_images2, silent):
    updated_images = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_image = {executor.submit(process_image, image_data, silent): image_data for image_data in new_images2}
        for future in concurrent.futures.as_completed(future_to_image):
            updated_images.append(future.result())
    
    return updated_images


In [8]:
new_images22 = [
    ("cropped_image0.jpg", (62, 237, 212, 38)),
    ("cropped_image1.jpg", (300, 300, 150, 150)),
    ("cropped_image2.jpg", (4, 237, 212, 38)),
    ("D://Retinafacetest/ms.jpg", (4, 237, 212, 38)),
    ("D://Retinafacetest/dar.jpg", (4, 237, 212, 38)),
    ("D://Retinafacetest/three1.jpg", (4, 237, 212, 38)),
    ("D://Retinafacetest/h.jpg", (4, 237, 212, 38)),
    
    
    
]
updated_images22 = FaceRecPP(new_images22, silent=False)
print(updated_images22)

[('cropped_image1.jpg', (300, 300, 150, 150), ['Customer4', 3, 41, 210, 210, 213, 251, 0.008378336968868916]), ('cropped_image0.jpg', (62, 237, 212, 38), ['Customer1', 28, 87, 162, 162, 190, 249, 0.00016068678009617532]), ('D://Retinafacetest/dar.jpg', (4, 237, 212, 38), ['Customer2', 0, 0, 474, 315, 474, 315, 0.04721419451671882]), ('cropped_image2.jpg', (4, 237, 212, 38), ['Customer', 0, 0, 0, 0, 0, 0, 0]), ('D://Retinafacetest/three1.jpg', (4, 237, 212, 38), ['Customer', 0, 0, 0, 0, 0, 0, 0]), ('D://Retinafacetest/h.jpg', (4, 237, 212, 38), ['Ali', 0, 0, 574, 685, 574, 685, 0.01307410502027595]), ('D://Retinafacetest/ms.jpg', (4, 237, 212, 38), ['Customer', 0, 0, 0, 0, 0, 0, 0])]


In [6]:

def load_and_preprocess_image(image_path):
    # Load image using TensorFlow
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=3)  # Ensure 3 color channels (RGB)
    # Preprocess image (resize, normalize, etc.)
    image = tf.image.resize(image, [224, 224])  # Example resize to 224x224
    image = image / 255.0  # Normalize pixel values to [0, 1]
    return image

def batch_process_images(image_paths, showlog):
    images = [load_and_preprocess_image(image_path) for image_path in image_paths]
    images_tensor = tf.convert_to_tensor(images)

    # Perform batch processing with TensorFlow
    # Assuming you have a TensorFlow model for face detection
    model = ...  # Load your TensorFlow face detection model
    predictions = model.predict(images_tensor)

    results = []
    for image_path, prediction in zip(image_paths, predictions):
        if showlog:
            print(f"\n----------------------------------------------\n")
            print(f"Image {image_path} : \nPrediction: {prediction}\n")
            print("\n----------------------------------------------\n")
        results.append((image_path, prediction))
    return results

def FaceRecPP(new_images2, showlog):
    image_paths = [image_path for image_path, _ in new_images2]
    coords_list = [coords for _, coords in new_images2]

    # Batch process all images
    batch_results = batch_process_images(image_paths, showlog)

    updated_images = []
    for (image_path, coords), (pred_path, prediction) in zip(new_images2, batch_results):
        updated_images.append((image_path, coords, prediction))
    
    return updated_images


In [7]:
new_images222 = [
    ("cropped_image0.jpg", (62, 237, 212, 38)),
    ("cropped_image1.jpg", (300, 300, 150, 150)),
    ("cropped_image2.jpg", (4, 237, 212, 38)),
    ("D://Retinafacetest/ms.jpg", (4, 237, 212, 38)),
    ("D://Retinafacetest/dar.jpg", (4, 237, 212, 38)),
    ("D://Retinafacetest/three1.jpg", (4, 237, 212, 38)),
    ("D://Retinafacetest/h.jpg", (4, 237, 212, 38)),
    
    
    
]
showlog = True
updated_images222 = FaceRecPP(new_images222, showlog)
print(updated_images222)

AttributeError: 'ellipsis' object has no attribute 'predict'

In [62]:
# finding=FaceRec(showlog=False)
# chatgpt format 
max_lengths = [max(len(str(item)) for item in data_column) for data_column in zip(*updated_images2)]

for row in updated_images2:
    formatted_row = " | ".join(f"{str(item):<{max_length+2}}" for item, max_length in zip(row, max_lengths))
    print(formatted_row)

# print(f"name= {name}")

cropped_image0.jpg   | (62, 237, 212, 38)     | ['Noore', 28, 87, 162, 162, 190, 249, 0.2882234397537323]    
cropped_image1.jpg   | (300, 300, 150, 150)   | ['hamza1', 3, 41, 210, 210, 213, 251, 0.30360839521578475]   
cropped_image2.jpg   | (4, 237, 212, 38)      | ['aboud', -11, 99, 251, 251, 240, 350, 0.01301719583403782]  


In [61]:

for i,index in enumerate(updated_images2):
    
    print(f"{index}")

('cropped_image0.jpg', (62, 237, 212, 38), ['Noore', 28, 87, 162, 162, 190, 249, 0.2882234397537323])
('cropped_image1.jpg', (300, 300, 150, 150), ['hamza1', 3, 41, 210, 210, 213, 251, 0.30360839521578475])
('cropped_image2.jpg', (4, 237, 212, 38), ['aboud', -11, 99, 251, 251, 240, 350, 0.01301719583403782])


# 